In [5]:
pip install polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 37.9 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import lightgbm as lgb
from lightgbm import LGBMClassifier, Dataset, cv, train, early_stopping
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import __version__ as sklearn_version
from packaging import version
from multiprocessing import cpu_count
import numpy as np
import shap
import matplotlib.pyplot as plt
import polars as pl
import itertools
import seaborn as sns

# Feature preprocessing 

In [29]:
### data preparation
"""
try:
    comorbidities = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_comorbidity.csv')
except FileNotFoundError:
    print("Incorrect path to comorbidities file or comorbidities dataframe manipulation section not run first.")
"""
"""
try:
    adi_scores = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_adi_scores.csv')
except FileNotFoundError:
    print("Incorrect path to ADI scores file.")
try:
    labs = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_labs.csv')
except FileNotFoundError:
    print("Incorrect path to labs file.")
try:
    vitals = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_vitals.csv')
except FileNotFoundError:
    print("Incorrect path to vitals file.")
try:
    prior_infecting_organisms = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_culture_prior_infecting_organism.csv')
except FileNotFoundError:
    print("Incorrect path to prior infecting organisms file.")
try:
    subtype_exposure = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_antibiotic_subtype_exposure.csv')
except FileNotFoundError:
    print("Incorrect path to subtype exposure file.")
try:
    demographics = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_demographics.csv')
except FileNotFoundError:
    print("Incorrect path to demographics file.")
"""
try:
    prior_med = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_prior_med.csv')
except FileNotFoundError:
    print("Incorrect path to prior medications file.")
"""
try:
    prior_procedures = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_priorprocedures.csv')
except FileNotFoundError:
    print("Incorrect path to prior procedures file.")
try:
    ward_info = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_ward_info.csv')
except FileNotFoundError:
    print("Incorrect path to ward info file.")
try:
    microbial_resistance = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_microbial_resistance.csv')
except FileNotFoundError:
    print("Incorrect path to microbial resistance file.")
try:
    cohort = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_cohort.csv')
except FileNotFoundError:
    print("Incorrect path to cohort file.")
try:
    antibiotic_class_exposure = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_antibiotic_class_exposure.csv')
except FileNotFoundError:
    print("Incorrect path to antibiotic class exposure file.")
try:
    implied_susceptibility = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_implied_susceptibility.csv')
except FileNotFoundError:
   print("Incorrect path to implied susceptibility file.")
try:
    nursing_home_visits = pd.read_csv('/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_nursing_home_visits.csv')
except FileNotFoundError:
    print("Incorrect path to nursing home visits file.")
"""

'\ntry:\n    prior_procedures = pd.read_csv(\'/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_priorprocedures.csv\')\nexcept FileNotFoundError:\n    print("Incorrect path to prior procedures file.")\ntry:\n    ward_info = pd.read_csv(\'/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_ward_info.csv\')\nexcept FileNotFoundError:\n    print("Incorrect path to ward info file.")\ntry:\n    microbial_resistance = pd.read_csv(\'/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_microbial_resistance.csv\')\nexcept FileNotFoundError:\n    print("Incorrect path to microbial resistance file.")\ntry:\n    cohort = pd.read_csv(\'/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_cohort.csv\')\nexcept FileNotFoundError:\n    print("Incorrect path to cohort file.")\ntry:\n    antibiotic_class_exposure = pd.read_csv(\'/Users/fa/Documents/antimicrobial-susceptibility/microbiology_cultures_antibiotic_class_exposure.csv\

# only in patients 

In [6]:
cohort

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,ordering_mode,culture_description,was_positive,organism,antibiotic,susceptibility
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,Inpatient,URINE,1,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible
1,JC1713666,131300064625,697566032,2020-12-27 00:40:00+00:00,Inpatient,URINE,1,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible
2,JC1669304,131272997044,620809641,2019-07-02 19:54:00+00:00,Inpatient,BLOOD,1,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible
3,JC1697441,131208305006,510635146,2016-12-01 13:59:00+00:00,Inpatient,URINE,1,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible
4,JC600786,131344486993,834128837,2022-11-18 19:35:00+00:00,Outpatient,URINE,1,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible
...,...,...,...,...,...,...,...,...,...,...
2241045,JC2809846,131354918630,878019747,2023-05-08 07:00:00+00:00,Outpatient,URINE,0,Null,Null,Null
2241046,JC671032,15528538,325819013,2008-06-04 12:50:00+00:00,Inpatient,BLOOD,0,Null,Null,Null
2241047,JC598858,131187132937,494883388,2016-04-29 21:08:00+00:00,Outpatient,URINE,0,Null,Null,Null
2241048,JC1698849,131263698251,593608493,2019-01-15 10:47:00+00:00,Inpatient,BLOOD,0,Null,Null,Null


In [7]:
cohort = cohort[cohort['was_positive'] == 1]
cohort = cohort[cohort['ordering_mode'] == 'Inpatient']
cohort = cohort.drop(columns = ['ordering_mode', 'was_positive'])
cohort['year'] = [time[:4] for time in cohort['order_time_jittered_utc']]
cohort=cohort[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','order_time_jittered_utc','culture_description','year']]
cohort.drop_duplicates(inplace=True)

In [8]:
implied_susceptibility=implied_susceptibility[(implied_susceptibility.organism!='Null')&(implied_susceptibility.antibiotic!='Null')]
implied_susceptibility.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,organism,antibiotic,susceptibility,implied_susceptibility
0,JC2673551,131337997983,831009441,ENTEROBACTER CLOACAE COMPLEX,imipenem,Null,Susceptible
1,JC616267,131007285560,358749950,ENTEROBACTER CLOACAE,imipenem,Null,Null
2,JC517471,131024758159,420255585,ENTEROBACTER ASBURIAE,doripenem,Null,Null
3,JC1931399,131200817434,512398445,MYCOBACTERIUM AVIUM COMPLEX,Clarithromycin,Susceptible,Null
4,JC605042,131280233261,642349435,KLEBSIELLA OXYTOCA,Ceftolozane/Tazobactam,Susceptible,Null


In [9]:

implied_susceptibility=implied_susceptibility[(implied_susceptibility.susceptibility!='Null')|(implied_susceptibility.implied_susceptibility!='Null')]
implied_susceptibility

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,organism,antibiotic,susceptibility,implied_susceptibility
0,JC2673551,131337997983,831009441,ENTEROBACTER CLOACAE COMPLEX,imipenem,Null,Susceptible
3,JC1931399,131200817434,512398445,MYCOBACTERIUM AVIUM COMPLEX,Clarithromycin,Susceptible,Null
4,JC605042,131280233261,642349435,KLEBSIELLA OXYTOCA,Ceftolozane/Tazobactam,Susceptible,Null
5,JC629724,131351986580,879371566,MUCOID PSEUDOMONAS AERUGINOSA,Ceftolozane/Tazobactam,Susceptible,Null
6,JC3257556,131010622629,373718233,KLEBSIELLA PNEUMONIAE,Trimethoprim/Sulfamethoxazole,Susceptible,Null
...,...,...,...,...,...,...,...
1978726,JC1968782,131346538225,887998404,ESCHERICHIA COLI,Amoxicillin/Clavulanic Acid,Susceptible,Null
1978727,JC1891401,131188090635,495913749,ESCHERICHIA COLI,Amoxicillin/Clavulanic Acid,Susceptible,Null
1978728,JC680569,131185719166,493570373,ESCHERICHIA COLI,Amoxicillin/Clavulanic Acid,Susceptible,Null
1978729,JC1242317,131213796632,514827508,ESCHERICHIA COLI,Amoxicillin/Clavulanic Acid,Susceptible,Null


In [10]:
#implied_susceptibility = implied_susceptibility[implied_susceptibility['antibiotic'].isin(['Cefazolin', 'Ceftriaxone', 'Cefepime', 'Piperacillin/Tazobactam', 'Ciprofloxacin'])]

implied_susceptibility.loc[
    implied_susceptibility['implied_susceptibility'] == 'Null', 
    'implied_susceptibility'
] = implied_susceptibility['susceptibility']
implied_susceptibility=implied_susceptibility[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','organism','antibiotic','implied_susceptibility']]
implied_susceptibility=implied_susceptibility[implied_susceptibility.implied_susceptibility.isin(['Susceptible', 'Resistant','Non Susceptible'])]
implied_susceptibility.loc[implied_susceptibility.implied_susceptibility=='Non Susceptible',
                           'implied_susceptibility'
                           ]='Resistant'
implied_susceptibility.implied_susceptibility.unique()

array(['Susceptible', 'Resistant'], dtype=object)

In [11]:
df = cohort.merge(implied_susceptibility, on = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'], how = 'left')
df = df[df['implied_susceptibility'].isin(['Susceptible', 'Resistant'])]
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant


In [12]:
df = df.merge(demographics, on = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'], how = 'left')

df.loc[df.age=='Null','age']=None
df.loc[df.gender=='Null','gender']=None
df.loc[df.gender=='0','gender']=0
df.loc[df.gender=='1','gender']=1

df.loc[df.age=='18–24 years','age']=1
df.loc[df.age=='25–34 years','age']=2
df.loc[df.age=='35–44 years','age']=3
df.loc[df.age=='45-54 years','age']=4
df.loc[df.age=='55-64 years','age']=5
df.loc[df.age=='65-74 years','age']=6
df.loc[df.age=='75-84 years','age']=7
df.loc[df.age=='85-89 years','age']=8
df.loc[df.age=='above 90','age']=9
df['age']=df['age'].astype(int)
df.age.unique()

array([7, 3, 6, 5, 4, 9, 8, 2, 1])

In [13]:
ward_info=ward_info[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','hosp_ward_IP','hosp_ward_OP','hosp_ward_ER','hosp_ward_ICU']]
ward_info.drop_duplicates(inplace=True)
for col in ward_info.columns.values:
    ward_info.loc[ward_info[col]=='Null',col]=None
ward_info.hosp_ward_IP.unique()

array([0., 1.])

In [14]:
adi_scores=adi_scores[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','adi_score']]
adi_scores.drop_duplicates(inplace=True)
for col in adi_scores.columns.values:
    adi_scores.loc[adi_scores[col]=='Null',col]=None
adi_scores.adi_score.unique()

array([None, '1', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100],
      dtype=object)

In [15]:
df = df.merge(ward_info, on = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'], how = 'left')
df = df.merge(adi_scores, on = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'], how = 'left')
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,gender,hosp_ward_IP,hosp_ward_OP,hosp_ward_ER,hosp_ward_ICU,adi_score
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,0,1.0,0.0,1.0,0.0,2
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,0,1.0,0.0,1.0,0.0,2
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,0,1.0,0.0,1.0,0.0,2
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,0,1.0,0.0,1.0,0.0,2
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,0,1.0,0.0,1.0,0.0,2


In [16]:
for col in vitals.columns.values:
    if not col in ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']:
        vitals.loc[vitals[col]=='Null',col]=None
        vitals[col]=vitals[col].astype('float')
vitals.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,Q25_heartrate,Q75_heartrate,median_heartrate,Q25_resprate,Q75_resprate,median_resprate,Q25_temp,...,first_diasbp,last_diasbp,last_sysbp,first_sysbp,last_temp,first_temp,last_resprate,first_resprate,last_heartrate,first_heartrate
0,JC2361817,131308278530,718222807,86.0,86.0,86.0,NaN,NaN,NaN,NaN,...,NaN,80.0,109.0,NaN,NaN,NaN,NaN,NaN,NaN,86.0
1,JC2219930,131318072067,748261410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,75.0,75.0,121.0,121.0,NaN,NaN,NaN,NaN,NaN,NaN
2,JC2455425,131256420868,574712812,70.0,70.0,70.0,NaN,NaN,NaN,NaN,...,69.0,NaN,NaN,106.0,NaN,NaN,NaN,NaN,70.0,NaN
3,JC1826078,131213257732,514463599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,90.0,90.0,120.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,JC1541734,131021676805,410407692,39.0,78.0,73.0,NaN,NaN,NaN,NaN,...,NaN,42.0,62.0,NaN,NaN,NaN,NaN,NaN,NaN,80.0


In [17]:
df = df.merge(vitals, on = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'], how = 'left')

In [18]:
nursing_home_visits=nursing_home_visits[nursing_home_visits.nursing_home_visit_culture>0]
nursing_home_visits['nursing_home_visits_within6month']=nursing_home_visits['nursing_home_visit_culture'].apply(lambda x: 1 if  x <= 180 else 0)
nursing_home_visits=nursing_home_visits[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','nursing_home_visits_within6month']]
nursing_home_visits = nursing_home_visits.groupby(['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']).agg(
    nursing_visits_within_6mo =('nursing_home_visits_within6month', sum),
).reset_index()
nursing_home_visits['nursing_visits_within_6mo']=nursing_home_visits['nursing_visits_within_6mo'].astype(int)

df = df.merge(nursing_home_visits, on = ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'], how = 'left')
df.head()

/var/folders/3b/31jnsf8150z57g6vqxxwj3vw0000gp/T/ipykernel_86976/2613113203.py:4: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  nursing_home_visits = nursing_home_visits.groupby(['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']).agg(


,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,...,last_diasbp,last_sysbp,first_sysbp,last_temp,first_temp,last_resprate,first_resprate,last_heartrate,first_heartrate,nursing_visits_within_6mo
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,...,61.0,121.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,...,61.0,121.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,...,61.0,121.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,...,61.0,121.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,...,61.0,121.0,NaN,NaN,NaN,NaN,18.0,NaN,NaN,NaN


In [19]:
prior_procedures=prior_procedures[(prior_procedures.procedure_description!='Null')&(prior_procedures.procedure_description.notna())]
prior_procedures=prior_procedures[(prior_procedures.procedure_time_to_culturetime.notna())&(prior_procedures.procedure_time_to_culturetime!='Null')]
prior_procedures=prior_procedures[(prior_procedures.procedure_time_to_culturetime>0)&(prior_procedures.procedure_time_to_culturetime<=180)]
procedures = prior_procedures.procedure_description.unique()

for procedure in procedures:
    prior_procedures[f"{procedure}_within_6mo"] = prior_procedures[prior_procedures['procedure_description'] == procedure]['procedure_time_to_culturetime'].apply(lambda x: 1 if x <= 180 else 0)
    

columns_to_sum = ['urethral_catheter_within_6mo','surgical_procedure_within_6mo','mechvent_within_6mo','cvc_within_6mo','parenteral_nutrition_within_6mo','dialysis_within_6mo']
agg_dict = {col: (col, 'sum') for col in columns_to_sum}
prior_procedures = prior_procedures.groupby(['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']).agg(**agg_dict).reset_index() 
prior_procedures=prior_procedures[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','urethral_catheter_within_6mo','surgical_procedure_within_6mo','mechvent_within_6mo','cvc_within_6mo','parenteral_nutrition_within_6mo','dialysis_within_6mo']]
prior_procedures.drop_duplicates(inplace=True)
for col in ['urethral_catheter_within_6mo','surgical_procedure_within_6mo','mechvent_within_6mo','cvc_within_6mo','parenteral_nutrition_within_6mo','dialysis_within_6mo']:
    prior_procedures[col]=prior_procedures[col].astype(int)
prior_procedures

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,urethral_catheter_within_6mo,surgical_procedure_within_6mo,mechvent_within_6mo,cvc_within_6mo,parenteral_nutrition_within_6mo,dialysis_within_6mo
0,JC1000129,131348917413,855365600,2,0,0,0,0,0
1,JC1000129,131354606122,876959985,3,0,0,0,0,0
2,JC1000129,131354606122,876959986,3,0,0,0,0,0
3,JC1000129,131359337847,894465321,1,0,0,0,0,0
4,JC1000143,131266832987,606756716,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
132516,JC999744,131208988464,511211017,0,1,0,0,0,0
132517,JC999754,131002270171,354437985,0,0,1,0,0,0
132518,JC999754,131002270171,354437986,0,0,1,0,0,0
132519,JC999793,131020075354,405267728,0,1,0,0,0,0


In [20]:
df = df.merge(prior_procedures, on =['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'], how = 'left')
df.iloc[:,42:]=df.iloc[:,42:].fillna(0)
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,...,first_resprate,last_heartrate,first_heartrate,nursing_visits_within_6mo,urethral_catheter_within_6mo,surgical_procedure_within_6mo,mechvent_within_6mo,cvc_within_6mo,parenteral_nutrition_within_6mo,dialysis_within_6mo
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,...,18.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,...,18.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,...,18.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,...,18.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,...,18.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
microbial_resistance=microbial_resistance[(microbial_resistance.organism!='Null')&(microbial_resistance.organism.notna())]
microbial_resistance=microbial_resistance[(microbial_resistance.antibiotic!='Null')&(microbial_resistance.antibiotic.notna())]
microbial_resistance=microbial_resistance[(microbial_resistance.resistant_time_to_culturetime>0)&(microbial_resistance.resistant_time_to_culturetime<=180)]
microbial_resistance=microbial_resistance[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','organism','antibiotic']]

# microbial resistance (game changer)
#microbial_resistance = microbial_resistance[microbial_resistance['antibiotic'].isin(['Cefazolin', 'Ceftriaxone', 'Cefepime', 'Piperacillin/Tazobactam', 'Ciprofloxacin'])]

# filter for top 10 most common microbes
microbial_resistance = microbial_resistance[microbial_resistance['organism'].isin(['ESCHERICHIA COLI', 'PSEUDOMONAS AERUGINOSA', 'MUCOID PSEUDOMONAS AERUGINOSA', 
                                                                                   'KLEBSIELLA PNEUMONIAE', 'ACHROMOBACTER XYLOSOXIDANS', 'STAPHYLOCOCCUS AUREUS', 
                                                                                   'PSEUDOMONAS AERUGINOSA (NON-MUCOID CF)', 'ENTEROCOCCUS SPECIES', 'ENTEROBACTER CLOACAE COMPLEX', 'PROTEUS MIRABILIS'])]

microbial_resistance.drop_duplicates(inplace=True)
microbial_resistance['organism_resistance_antibiotic'] = microbial_resistance['organism'] + '_' + microbial_resistance['antibiotic']
microbial_resistance=microbial_resistance[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','organism_resistance_antibiotic']]
microbial_resistance.drop_duplicates(inplace=True)

In [22]:
microbial_resistance = microbial_resistance.groupby(["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "organism_resistance_antibiotic"])\
              .size()\
              .unstack(fill_value=0)\
              .reset_index()
microbial_resistance.columns.name = None 
print(microbial_resistance)


         anon_id  pat_enc_csn_id_coded  order_proc_id_coded  \
0      JC1000129          131348917413            855365600   
1      JC1000129          131354606122            876959985   
2      JC1000129          131354606122            876959986   
3      JC1000129          131359337847            894465321   
4      JC1000169          131201261980            506588339   
...          ...                   ...                  ...   
61958   JC999695          131238748562            536694630   
61959   JC999695          131238748562            536694632   
61960   JC999695          131240032823            539891925   
61961   JC999695          131240032823            539891927   
61962   JC999695          131240032823            539891931   

       ACHROMOBACTER XYLOSOXIDANS_Amikacin  \
0                                        0   
1                                        0   
2                                        0   
3                                        0   
4            

In [23]:
df = df.merge(microbial_resistance, on = ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'], how = 'left')
df.iloc[:, 48:] = df.iloc[:, 48:].fillna(0)
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,...,STAPHYLOCOCCUS AUREUS_Cefazolin,STAPHYLOCOCCUS AUREUS_Ciprofloxacin,STAPHYLOCOCCUS AUREUS_Erythromycin,STAPHYLOCOCCUS AUREUS_Gentamicin,STAPHYLOCOCCUS AUREUS_Levofloxacin,STAPHYLOCOCCUS AUREUS_Linezolid,STAPHYLOCOCCUS AUREUS_Moxifloxacin,STAPHYLOCOCCUS AUREUS_Penicillin,STAPHYLOCOCCUS AUREUS_Rifampin,STAPHYLOCOCCUS AUREUS_Vancomycin
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
prior_infecting_organisms=prior_infecting_organisms[(prior_infecting_organisms.prior_infecting_organism_days_to_culutre<=180)&
                                                    (prior_infecting_organisms.prior_infecting_organism_days_to_culutre>0)]

prior_infecting_organisms=prior_infecting_organisms[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','prior_organism']]
prior_infecting_organisms.drop_duplicates(inplace=True)
prior_infecting_organisms

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,prior_organism
0,JC1000055,131007833415,360359154,Escherichia
4,JC1000129,131348917413,855365600,Escherichia
6,JC1000129,131359337847,894465321,Escherichia
8,JC1000129,131359337847,894465321,Proteus
11,JC1000169,131216209091,516543570,Escherichia
...,...,...,...,...
1084087,JC999190,131362324212,907799972,Escherichia
1084091,JC999190,131365356752,915878692,Klebsiella
1084097,JC999190,131365356752,915878693,Klebsiella
1084110,JC999396,131246456560,552045783,Staphylococcus


In [25]:
prior_infecting_organisms = prior_infecting_organisms.groupby(["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "prior_organism"])\
              .size()\
              .unstack(fill_value=0)\
              .reset_index()
prior_infecting_organisms.columns.name = None  
prior_infecting_organisms.columns = [f"prior_infected_{col}" if 3 <= i <= 19 else col for i, col in enumerate(prior_infecting_organisms.columns)]
prior_infecting_organisms = prior_infecting_organisms[prior_infecting_organisms.iloc[:, 3:20].gt(0).any(axis=1)]
print(prior_infecting_organisms)


          anon_id  pat_enc_csn_id_coded  order_proc_id_coded  \
0       JC1000055          131007833415            360359154   
1       JC1000083          131013906068            384652929   
2       JC1000129          131348917413            855365600   
3       JC1000129          131354606122            876959985   
4       JC1000129          131354606122            876959986   
...           ...                   ...                  ...   
105045   JC999695          131238748562            536694630   
105046   JC999695          131238748562            536694632   
105047   JC999695          131240032823            539891925   
105048   JC999695          131240032823            539891927   
105049   JC999695          131240032823            539891931   

        prior_infected_Acinetobacter  prior_infected_CONS  \
0                                  0                    0   
1                                  0                    0   
2                                  0            

In [26]:
df = df.merge(prior_infecting_organisms, on = ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'], how = 'left')
df.iloc[:, 79:] = df.iloc[:, 79:].fillna(0)
df.drop_duplicates(inplace=True)
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,...,prior_infected_Escherichia,prior_infected_Klebsiella,prior_infected_Morganella,prior_infected_Proteus,prior_infected_Providencia,prior_infected_Pseudomonas,prior_infected_Serratia,prior_infected_Staphylococcus,prior_infected_Stenotrophomonas,prior_infected_Streptococcus
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
prior_med=prior_med[(prior_med.medication_time_to_culturetime>0)&(prior_med.medication_time_to_culturetime<=180)]
prior_med=prior_med[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','medication_name']]
prior_med.drop_duplicates(inplace=True)
prior_med

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,medication_name
2,JC1005213,131242737476,546059312,Amikacin
6,JC1018201,131007178838,359199842,Ceftazidime
33,JC1021489,131327676051,777953629,Ceftazidime
40,JC1024438,131011766132,376285814,Amikacin
41,JC1024438,131013577021,383561655,Amikacin
...,...,...,...,...
1952136,JC992726,131033115545,439372834,Metronidazole
1952198,JC993612,131306005803,713974419,Metronidazole
1952210,JC994928,32767015,350394505,Metronidazole
1952237,JC996143,131280542195,643440579,Metronidazole


In [31]:
prior_med = prior_med.groupby(["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "medication_name"])\
              .size()\
              .unstack(fill_value=0)\
              .reset_index()
prior_med.columns.name = None 
prior_med.columns = [f"prior_med_{col}" if 3 <= i <= 23 else col for i, col in enumerate(prior_med.columns)]
prior_med = prior_med[prior_med.iloc[:, 3:23].gt(0).any(axis=1)]
print(prior_med)

          anon_id  pat_enc_csn_id_coded  order_proc_id_coded  \
0       JC1000129          131359337847            894465321   
1       JC1000143          131312409361            730719899   
2       JC1000441          131211600088            513233094   
3       JC1000492          131356383252            883656151   
4       JC1000492          131356383252            883656152   
...           ...                   ...                  ...   
130166   JC999695          131240032823            539891931   
130167   JC999744          131208988464            511211016   
130168   JC999744          131208988464            511211017   
130169   JC999893          131320319431            761441463   
130170   JC999908          131189316002            496947265   

        prior_med_Amikacin  prior_med_Ampicillin  prior_med_Aztreonam  \
0                        0                     0                    0   
1                        0                     0                    0   
2           

In [32]:
df = df.merge(prior_med, on = ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'],how='left')
df.iloc[:, 95:] = df.iloc[:, 95:].fillna(0)
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,...,prior_med_Colistin,prior_med_Ertapenem,prior_med_Gentamicin,prior_med_Isoniazid,prior_med_Levaquin,prior_med_Levofloxacin,prior_med_Linezolid,prior_med_Metronidazole,prior_med_Penicillin,prior_med_Vancomycin
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
antibiotic_class_exposure=antibiotic_class_exposure[(antibiotic_class_exposure.time_to_culturetime>0)&(antibiotic_class_exposure.time_to_culturetime<=180)]
antibiotic_class_exposure=antibiotic_class_exposure[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','antibiotic_class']]
antibiotic_class_exposure.drop_duplicates(inplace=True)
antibiotic_class_exposure

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,antibiotic_class
5,JC1409554,131022617974,413869070,Beta Lactam
8,JC1285316,131283292381,652260365,Beta Lactam
25,JC1324289,131013992116,386702169,Fluoroquinolone
89,JC1579122,131027230452,426958539,Beta Lactam
90,JC1442941,131311181146,728547454,Beta Lactam
...,...,...,...,...
12018842,JC1349355,131358297968,890540152,Glycopeptide
12018903,JC6513275,131360944888,900381391,Glycopeptide
12018914,JC559676,131356777585,884929042,Glycopeptide
12018927,JC6346832,131352755932,869760638,Glycopeptide


In [34]:
antibiotic_class_exposure = antibiotic_class_exposure.groupby(["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "antibiotic_class"])\
              .size()\
              .unstack(fill_value=0)\
              .reset_index()
antibiotic_class_exposure.columns.name = None  
antibiotic_class_exposure.columns = [f"prior_abx_class_{col}" if 3 <= i else col for i, col in enumerate(antibiotic_class_exposure.columns)]
print(antibiotic_class_exposure)

          anon_id  pat_enc_csn_id_coded  order_proc_id_coded  \
0       JC1000129          131348917413            855365600   
1       JC1000129          131354606122            876959985   
2       JC1000129          131354606122            876959986   
3       JC1000129          131359337847            894465321   
4       JC1000143          131312409361            730719899   
...           ...                   ...                  ...   
324244   JC999842          131012467494            379835495   
324245   JC999842          131192863784            500208763   
324246   JC999893          131320319431            761441463   
324247   JC999908          131189316002            496947265   
324248   JC999992          131189547541            497201103   

        prior_abx_class_Aminoglycoside  prior_abx_class_Ansamycin  \
0                                    0                          0   
1                                    0                          0   
2                       

In [35]:
df = df.merge(antibiotic_class_exposure, on = ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'], how = 'left')
df.iloc[:, 115:] = df.iloc[:, 115:].fillna(0)
df.head()

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,order_time_jittered_utc,culture_description,year,organism,antibiotic,implied_susceptibility,age,...,prior_abx_class_Glycopeptide,prior_abx_class_Macrolide Lincosamide,prior_abx_class_Monobactam,prior_abx_class_Nitrofuran,prior_abx_class_Nitroimidazole,prior_abx_class_Oxazolidinone,"prior_abx_class_Polymyxin, Lipopeptide",prior_abx_class_Sulfonamide,prior_abx_class_Tetracycline,prior_abx_class_Urinary Antiseptic
0,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Amikacin,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Cefepime,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ertapenem,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Meropenem,Susceptible,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,JC2744063,131368600230,928257722,2023-12-23 22:29:00+00:00,URINE,2023,KLEBSIELLA PNEUMONIAE,Ampicillin,Resistant,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df = df.merge(labs, on = ['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded'], how = 'left')
df.head()

In [36]:
df.to_csv('Model1_AllABX_fixedTime0.csv',index=False)
#df=pd.read_csv('Model1_AllABX.csv')

In [ ]:
comorbidities=comorbidities[((comorbidities.comorbidity_component_start_days_culture<=180) & (comorbidities.comorbidity_component_start_days_culture>=0)) &
                            ((comorbidities.comorbidity_component_end_days_culture<0) | (comorbidities.comorbidity_component_end_days_culture.isna()))]


comorbidities=comorbidities[(comorbidities.comorbidity_component!='Null')&(comorbidities.comorbidity_component.notna())]
comorbidities=comorbidities[["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "comorbidity_component"]]
comorbidities.drop_duplicates(inplace=True)

comorbidities = comorbidities.groupby(["anon_id", "pat_enc_csn_id_coded", "order_proc_id_coded", "comorbidity_component"])\
              .size()\
              .unstack(fill_value=0)\
              .reset_index()
comorbidities.columns.name = None 
print(comorbidities)

In [ ]:
df = df.merge(comorbidities, on = ['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'], how = 'left')
#df.iloc[:, 189:] = df.iloc[:, 189:].fillna(0)
#df

In [ ]:
df.order_proc_id_coded.nunique()

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
df.to_csv('Model1_with_Comorbidity_components.csv',index=False)